# Met Eireann wind speed analysis
### Programming for Data Analytics Project
#### Author: Vanessa de Oliveira Lyra


In [10]:
# Importing libraries
import pandas as pd

In [ ]:
# Reading .csv file into a dataframe using semicolon separator
df = pd.read_csv("StationDetails.csv", sep=";")
print(df.head())

  county,station name,name,height(m),easting,northing,latitude,longitude,open year,close year
0  Antrim,5880,LH_RATHLIN_WEST,10,309200,451800,5...                                         
1  Carlow,4415,TULLOW (Waterworks),76,284700,1734...                                         
2  Carlow,2414,BORRIS G.S.,85,272400,150700,52.60...                                         
3  Carlow,1214,CARLOW (SUGAR FACTORY),58,272200,1...                                         
4  Carlow,115,HACKETSTOWN RECTORY,182,297600,1805...                                         


https://stackoverflow.com/questions/18039057/pandas-parser-cparsererror-error-tokenizing-data #Specify delimiter to read data

End